# Tutorial Panggilan Fungsi Selari Phi-4 Mini ONNX

Notebook ini menunjukkan cara menggunakan Phi-4 Mini dengan ONNX Runtime GenAI untuk panggilan fungsi selari. Panggilan fungsi membolehkan model secara bijak menggunakan alat dan API luaran berdasarkan permintaan pengguna.

## Gambaran Keseluruhan

Dalam tutorial ini, anda akan belajar cara:
- Menyediakan Phi-4 Mini dengan ONNX Runtime GenAI
- Menentukan skema fungsi untuk tempahan penerbangan dan hotel
- Menggunakan penjanaan berpandu dengan tatabahasa Lark untuk output berstruktur
- Melaksanakan panggilan fungsi selari untuk senario tempahan perjalanan yang kompleks

## Prasyarat

Sebelum menjalankan notebook ini, pastikan anda:
- Memuat turun model Phi-4 Mini ONNX
- Memasang pakej `onnxruntime-genai`
- Memahami asas konsep panggilan fungsi


## Langkah 1: Import Perpustakaan Diperlukan

Pertama, kita akan mengimport perpustakaan yang diperlukan untuk pelaksanaan pemanggilan fungsi kita.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Langkah 2: Persediaan dan Konfigurasi Model

Sekarang kita akan mengkonfigurasi model Phi-4 Mini ONNX. Pastikan untuk menggantikan laluan dengan direktori model sebenar anda.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Langkah 3: Konfigurasi Parameter Penjanaan

Tetapkan parameter penjanaan untuk mengawal tingkah laku output model. Tetapan ini memastikan respons yang deterministik dan fokus untuk pemanggilan fungsi.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Langkah 4: Tentukan Fungsi yang Tersedia

Di sini kita menentukan fungsi yang boleh dipanggil oleh pembantu AI kita. Dalam contoh ini, terdapat dua fungsi berkaitan perjalanan:
1. **booking_flight_tickets**: Untuk tempahan penerbangan antara lapangan terbang
2. **booking_hotels**: Untuk tempahan penginapan hotel

Definisi fungsi mengikuti format skema pemanggilan fungsi OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Langkah 5: Fungsi Pembantu Penjanaan Tatabahasa

Fungsi pembantu ini menukar definisi fungsi kita ke format tatabahasa Lark, yang digunakan oleh ONNX Runtime GenAI untuk penjanaan berpandu. Ini memastikan model menghasilkan panggilan fungsi yang sah dalam format JSON yang betul.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Langkah 6: Uji Penjanaan Tatabahasa

Mari kita uji fungsi penjanaan tatabahasa kita untuk melihat bagaimana ia menukar definisi alat kita ke dalam format yang betul.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Langkah 7: Sediakan Prompt Sistem dan Penjana

Sekarang kita akan mencipta prompt sistem yang memberitahu model tentang alat yang tersedia dan menyediakan penjana dengan parameter penjanaan berpandu.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Langkah 8: Inisialisasi Penjana dengan Penjanaan Berpanduan

Sekarang kita akan mencipta penjana dengan parameter yang telah dikonfigurasi dan menggunakan tatabahasa Lark untuk penjanaan berpanduan.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Langkah 9: Uji Panggilan Fungsi Selari

Sekarang mari kita uji tetapan kita dengan senario tempahan perjalanan yang kompleks yang memerlukan panggilan kepada pelbagai fungsi.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Menjana Panggilan Fungsi

Model kini akan menjana panggilan fungsi berstruktur berdasarkan permintaan pengguna. Dengan bantuan penjanaan berpandu, output akan berada dalam format JSON yang sah dan boleh dilaksanakan secara langsung.

**Output Dijangka**: Model sepatutnya menjana panggilan fungsi untuk:
1. `booking_flight_tickets` dengan butiran Beijing (PEK) ke Paris (CDG)
2. `booking_hotels` dengan butiran penginapan di Paris

Jalankan sel di bawah untuk melihat penjanaan secara langsung:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Kesimpulan

🎉 **Tahniah!** Anda telah berjaya melaksanakan panggilan fungsi selari dengan Phi-4 Mini menggunakan ONNX Runtime GenAI.

### Apa Yang Anda Pelajari:

1. **Persediaan Model**: Cara mengkonfigurasi Phi-4 Mini dengan ONNX Runtime GenAI
2. **Definisi Fungsi**: Cara mendefinisikan skema fungsi untuk panggilan fungsi AI
3. **Penjanaan Berpanduan**: Cara menggunakan tatabahasa Lark untuk menghasilkan output yang berstruktur
4. **Panggilan Fungsi Selari**: Cara mengurus permintaan kompleks yang memerlukan pelbagai panggilan fungsi

### Manfaat Utama:

- ✅ **Output Berstruktur**: Penjanaan berpanduan memastikan panggilan fungsi JSON yang sah
- ✅ **Pemprosesan Selari**: Mengurus pelbagai panggilan fungsi dalam satu permintaan
- ✅ **Prestasi Tinggi**: ONNX Runtime menyediakan inferens yang dioptimumkan
- ✅ **Skema Fleksibel**: Mudah untuk menambah atau mengubah definisi fungsi

### Sumber:

- [Dokumentasi Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Dokumentasi ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Amalan Terbaik Panggilan Fungsi](https://platform.openai.com/docs/guides/function-calling)



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
